In [1]:
import tensorflow as tf
import numpy as np
np.set_printoptions(threshold=np.inf)
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow import estimator
import pandas as pd
tf.reset_default_graph()

In [2]:
def normalize(df):    
    return (df - df.min()) / (df.max() - df.min())


def denormalize(df,norm_data):    
    return (norm_data * (df.max() - df.min())) + df.min()


def preProcessForTrain(sensor_no):
    sensor=str(sensor_no)
    df = pd.read_csv('./data/finalData/'+sensor+'.csv')
    df['out']=df['out']*1
    df = df.dropna(inplace=False)
    msk = np.random.rand(len(df)) < 0.6
    df_train = df[msk]    
    df_test = df[~msk]
    X_train = normalize(df_train.drop(['out'],axis=1)).values
    y_train = df_train['out'].values
    X_test = normalize(df_test.drop(['out'],axis=1)).values
    y_test = df_test['out'].values
    return X_train, y_train, X_test, y_test


def preProcessForTest(sensor_no):
    sensor=str(sensor_no)
    df = pd.read_csv('./data/finalData/'+sensor+'.csv')
    df['out']=df['out']*1
    df = df.dropna(inplace=False)
    msk = np.random.rand(len(df)) < 1
    df_test = df[msk]
    X_test = normalize(df_test.drop(['out'],axis=1)).values
    y_test = df_test['out'].values
    return X_test, y_test


def DNNmodel():
    feat_cols=[tf.feature_column.numeric_column('x', shape=[961])]
    return tf.estimator.DNNClassifier(feature_columns=feat_cols, 
                                      n_classes=2,
                                      hidden_units=[1000,1000,1000,1000,1000,1000], 
                                      activation_fn=tf.nn.relu,
                                      optimizer=tf.train.GradientDescentOptimizer(0.0001),
                                      model_dir='./model')


def training(X_train, y_train):
    feat_cols=[tf.feature_column.numeric_column('x', shape=[961])]
    deep_model=DNNmodel()
    input_fn = estimator.inputs.numpy_input_fn(x={'x':X_train}, 
                                           y=y_train,
                                           shuffle= True,
                                           num_epochs=5000,
                                           batch_size=100)
    return deep_model.train(input_fn=input_fn, steps=5000)


def evaluate(X_test, y_test):   
    input_fn_eval = estimator.inputs.numpy_input_fn( x = {'x':X_test},
                                                y =  y_test,
                                                shuffle = False)
    deep_model=DNNmodel()
    
    return dict(deep_model.evaluate(input_fn=input_fn_eval))


def predict(X_test):
    
    input_fn_eval = estimator.inputs.numpy_input_fn( x = {'x':X_test},
                                                shuffle = False)
    deep_model = DNNmodel()
    preds=list(deep_model.predict(input_fn=input_fn_eval))
    predictions = [p['class_ids'][0] for p in preds]
    pred = np.asarray(predictions)
    return pred    


def checkAccuracy(pred, y_test):
    f = pred == y_test
    T = len(f[f == True])
    F = len(f[f == False])
    error = F/(T+F) * 100
    return T, F, error 

In [30]:
sites = [#{'site':'BARC',                       'lat':19.0436,  'lng':72.9197, 'srNo':},
         {'site':'Aggarwal College Ballabgarh', 'lat':28.3317,  'lng':77.3344, 'srNo': 1613},
         {'site':'Gauhati University',          'lat':26.1551,  'lng':91.6627, 'srNo': 1629},
         {'site':'GZRSC, Aizawl',               'lat':23.7398,  'lng':92.7223, 'srNo': 1617},
         #{'site':'IIG Vizag',                  'lat':17.68,    'lng':83.32,   'srNo':},
         {'site':'IOP Bhubaneshwar',            'lat':20.3091,  'lng':85.8309, 'srNo': 1631},
         {'site':'Kashmir University',          'lat':34.12796, 'lng':74.8365, 'srNo': 1622},
         {'site':'Manipur University',          'lat':24.7507,  'lng':93.9291, 'srNo': 1612},
         {'site':'Mizoram University',          'lat':23.735,   'lng':92.6671, 'srNo': 1614},
         {'site':'Nagaland University',         'lat':25.7247,  'lng':94.0955, 'srNo': 1615},
         {'site':'NEHU Shillong',               'lat':25.6089,  'lng':91.8999, 'srNo': 1630},
         {'site':'NIT Rourkela',                'lat':22.2533,  'lng':84.9011, 'srNo': 1601},
         {'site':'SGRC Shillong',               'lat':25.61,    'lng':91.896,  'srNo': 1512},
         {'site':'Silchar Assam',               'lat':24.6919,  'lng':92.7503, 'srNo': 1607},
         {'site':'SINP Kolkata',                'lat':22.5997,  'lng':88.42,   'srNo': 1627},
         {'site':'IIT Roorkee',                 'lat':29.8649,  'lng':77.8966, 'srNo': 1610},
        ]

In [42]:
preds = []
for i in sites:
    selectedSite = i['srNo']
    X_test, y_test = preProcessForTest(selectedSite)
    pred = evaluate(X_test, y_test)
    pred['site'] = i['srNo']
    preds.append(pred)
df1=pd.DataFrame(preds)
print(df1)
df1.to_csv('./Alldata/AllData.csv') 

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_master': '', '_model_dir': './model', '_session_config': None, '_keep_checkpoint_max': 5, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f894cc4bcf8>, '_service': None, '_keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_num_worker_replicas': 1, '_save_summary_steps': 100, '_task_type': 'worker', '_save_checkpoints_secs': 600, '_task_id': 0, '_tf_random_seed': None, '_train_distribute': None, '_num_ps_replicas': 0, '_evaluation_master': '', '_global_id_in_cluster': 0, '_log_step_count_steps': 100, '_save_checkpoints_steps': None}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-06-12:48:08
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model/model.ckpt-20005
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evalu

In [37]:
! pwd


/home/abhishek/2018/trainee/eventPredictor


In [28]:
selectedSite = 1601
X_test, y_test = preProcessForTest(selectedSite)
pred = evaluate(X_test, y_test)
df1=pd.DataFrame.from_dict(pred, orient='index')
df1.to_csv('Alldata/'+str(selectedSite)+'.csv')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_master': '', '_model_dir': './model', '_session_config': None, '_keep_checkpoint_max': 5, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f895006dac8>, '_service': None, '_keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_num_worker_replicas': 1, '_save_summary_steps': 100, '_task_type': 'worker', '_save_checkpoints_secs': 600, '_task_id': 0, '_tf_random_seed': None, '_train_distribute': None, '_num_ps_replicas': 0, '_evaluation_master': '', '_global_id_in_cluster': 0, '_log_step_count_steps': 100, '_save_checkpoints_steps': None}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-06-12:41:17
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model/model.ckpt-20005
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evalu